In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import math
import torch
import pickle
import numpy as np
import pandas as pd
#import seaborn as sns

import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.cluster import KMeans
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import joblib

import holidays
us_holidays = holidays.country_holidays('US')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

directory = '/content/drive/MyDrive/Thesis'
data_dir = directory + "/Data"
models_dir = directory + "/models"

In [ ]:
from hyperopt.pyll.base import scope
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
!unzip /content/drive/MyDrive/Thesis/Data/gnn_cnn_2024-11-02_14_35_41.074099_output_embedding.zip

Archive:  /content/drive/MyDrive/Thesis/Data/gnn_cnn_2024-11-02_14_35_41.074099_output_embedding.zip
  inflating: gnn_cnn_2024-11-02 14_35_41.074099_output_embedding.pt  


In [ ]:
demand = pd.read_csv(f"{data_dir}/pre_processed_data.csv")

df = demand.sort_values(by=['started_at_year', 'started_at_month',
                            'started_at_day', 'started_at_hour'])
df = df.drop(columns=['Unnamed: 0', 'duration_sec']).reset_index(drop=True)
df['started_at_daily'] = pd.to_datetime(df['started_at_daily'])
df['started_at_hourly'] = pd.to_datetime(df['started_at_hourly'])

df = df[df['clusters']==1]
df = df[df['started_at_hourly'] >= pd.to_datetime("2023-01-01 00:00:00")].drop_duplicates(
    ['started_at_hourly', 'start_station_cluster', 'demand']).reset_index(drop=True)

In [ ]:
df['start_station_cluster'].max()

278

In [ ]:
earliest_trips = df.groupby('start_station_cluster').nth(0)
stations_operational_since_early = earliest_trips[(earliest_trips['started_at_month'] < 4)]['start_station_cluster']
stations_early_enough = df['start_station_cluster'].isin(stations_operational_since_early)
print((df[~stations_early_enough].shape[0]/df.shape[0])*100)
df = df[stations_early_enough].reset_index(drop=True)

1.9881767405363227


In [ ]:
CLUSTER_COLUMN = 'start_station_cluster'
DATETIME = "started_at_hourly"
DATE = "started_at_daily"

# Filling Gaps

In [ ]:
def fill_three_day_window(day, df, cluster):
    # Define the three-day window (previous day, current day, and next day)
    start_date = day - pd.Timedelta(days=1)
    end_date = day + pd.Timedelta(days=1)

    days_to_fill = []
    for i in range(3):
        candidate_day = start_date + pd.Timedelta(days=i)
        if candidate_day not in processed_dates:
            days_to_fill.append(candidate_day)
    #print(days_to_fill)
    # If all days overlap, skip filling this window
    if not days_to_fill:
        return None
    days_to_fill[-1] = days_to_fill[-1] - pd.Timedelta(hours=1)
    # Create a full hourly range only for the non-overlapping days
    full_range = pd.date_range(start=min(days_to_fill), end=max(days_to_fill), freq='h')


    # Reindex the DataFrame to this range and interpolate/forward-fill
    df_window = df.reindex(full_range)
    df_window['demand'] = df_window['demand'].fillna(0)
    #df_window['flag_added'] = df_window['flag_added'].fillna(True)
    df_window['start_station_cluster'] = cluster  # or use fillna()
    df_window['clusters'] = 1  # or use fillna()

    # Add these dates to the processed set
    new_days = [start_date + pd.Timedelta(days=i) for i in range(3)]
    new_days[-1] = new_days[-1] - pd.Timedelta(hours=1)

    processed_dates.update()

    return df_window

In [ ]:
def add_lags(df, target, identifier):

  df_res = pd.DataFrame()
  print(target)
  for ii in df[identifier].unique():
      df_current = df[df[identifier]==ii].copy()
      df_current.index = df_current['started_at_hourly']
      df_current.index = pd.to_datetime(df_current.index)
      target_map = df_current[target].to_dict()
      df_current[f"{target}_lag_1_h"] = (df_current.index - pd.Timedelta('1 hours')).map(target_map)
      df_current[f"{target}_lag_2_h"] = (df_current.index - pd.Timedelta('2 hours')).map(target_map)
      df_current[f"{target}_lag_24_h"] = (df_current.index - pd.Timedelta('24 hours')).map(target_map)
      if target == "demand":
          df_current[f"{target}_lag_1_h"] = df_current[f"{target}_lag_1_h"].fillna(0)
          df_current[f"{target}_lag_2_h"] = df_current[f"{target}_lag_2_h"].fillna(0)
          df_current[f"{target}_lag_24_h"] = df_current[f"{target}_lag_24_h"].fillna(0)
      else:
          df_current[f"{target}_lag_1_h"] = df_current[f"{target}_lag_1_h"].interpolate().fillna(0)
          df_current[f"{target}_lag_2_h"] = df_current[f"{target}_lag_2_h"].interpolate().fillna(0)
          df_current[f"{target}_lag_24_h"] = df_current[f"{target}_lag_24_h"].interpolate().fillna(0)
      df_res = pd.concat([df_res, df_current])
  return df_res

In [ ]:
df_concatenated = pd.DataFrame()
for i in df[CLUSTER_COLUMN].unique():

  df_current = df[df[CLUSTER_COLUMN]==i].copy()
  #df_current['flag_added'] = False
  df_current[DATETIME] = pd.to_datetime(df_current[DATETIME])
  df_current.set_index(DATETIME, inplace=True)
  unique_dates = df_current.index.normalize().unique()

  # Create an empty DataFrame to hold the result
  df_filled = pd.DataFrame()

  # Track days that have already been processed to avoid overlap
  processed_dates = set()
  #print(i)
  # Function to generate a three-day range and fill missing hours
  # Process each unique date
  for date in unique_dates:
      df_current_date = df_current[df_current[DATE] == date].copy()
      filled_window = fill_three_day_window(date, df_current_date, i)
      #print(filled_window.iloc[:, 0:7])
      #print()
      if filled_window is not None:
          df_filled = pd.concat([df_filled, filled_window])

  # Drop duplicates (if any) and sort the DataFrame
  df_filled = df_filled[~df_filled.index.duplicated(keep='first')].sort_index()
  df_concatenated = pd.concat([df_concatenated, df_filled])

del df_filled


# Datetime

In [ ]:
def create_datetime_features(input_df, column_name):

    input_df[column_name+'_hour'] = input_df[column_name].dt.hour
    input_df[column_name+'_quarter'] = input_df[column_name].dt.quarter
    input_df[column_name+'_month'] = input_df[column_name].dt.month
    input_df[column_name+'_year'] = input_df[column_name].dt.year
    input_df[column_name+'_week'] = input_df[column_name].dt.isocalendar().week
    input_df[column_name+'_day'] = input_df[column_name].dt.day
    input_df[column_name+'_dayofweek'] = input_df[column_name].dt.dayofweek

    return input_df

In [ ]:
def add_datetime(df, column_name="started_at"):
    conversion_dict_hourly = dict(year= df[f'{column_name}_year'],
                                  month=df[f'{column_name}_month'],
                                  day=  df[f'{column_name}_day'],
                                  hour= df[f'{column_name}_hour']
                                 )
    conversion_dict_daily = dict(year= df[f'{column_name}_year'],
                           month=df[f'{column_name}_month'],
                           day=  df[f'{column_name}_day']
                                 )
    df[f'{column_name}_hourly'] = pd.to_datetime(conversion_dict_hourly)

    df[f'{column_name}_daily'] = pd.to_datetime(conversion_dict_daily)

    return df

In [ ]:
df_concatenated = df_concatenated.reset_index().rename(columns={
    "index": "started_at_hourly"
})

In [ ]:
df_concatenated = add_datetime(create_datetime_features(
    df_concatenated.rename(columns={"started_at_hourly": "started_at"}).copy(), "started_at"
))
df_concatenated = df_concatenated.drop(columns=['started_at'])

In [ ]:
df_concatenated['is_holiday'] =  df_concatenated['started_at_hourly'].apply(lambda x: x in us_holidays)

# Weather Data

In [ ]:
weather_data = pd.read_csv(f"{data_dir}/weather_data.csv")

In [ ]:
weather_data = pd.read_csv(f"{data_dir}/weather_data.csv")
weather_data['started_at_hourly'] = pd.to_datetime(weather_data['started_at_hourly'])

median_code = weather_data["coco"].median()

weather_data["coco"] = weather_data["coco"].fillna(median_code).astype(int)



number_of_clusters = df['start_station_cluster'].nunique()

weather_data_time = weather_data.reset_index().rename(columns={
    "time": "started_at_hourly"
})
df_concatenated = df_concatenated.merge(weather_data_time[['temp', 'dwpt', "rhum", "prcp", "wdir", "wspd", "pres", "coco", "centroid", 'started_at_hourly']].rename(columns={
    "centroid":"clusters"
}), on=['clusters', "started_at_hourly"], how='left')

# Lags

In [ ]:
df_concatenated = df_concatenated.sort_values("started_at_hourly")
df_concatenated = add_lags(df_concatenated, "demand", identifier=CLUSTER_COLUMN)
df_concatenated = add_lags(df_concatenated, "temp", identifier=CLUSTER_COLUMN)
df_concatenated = add_lags(df_concatenated, "prcp", identifier=CLUSTER_COLUMN)
df_concatenated = add_lags(df_concatenated, "rhum", identifier=CLUSTER_COLUMN)
df_concatenated = add_lags(df_concatenated, "wspd", identifier=CLUSTER_COLUMN)
df_concatenated = add_lags(df_concatenated, "coco", identifier=CLUSTER_COLUMN)

demand
temp
prcp
rhum
wspd
coco


In [ ]:
df_concatenated = df_concatenated.query('~((started_at_hour>=2) & (started_at_hour<=5))').copy().reset_index(drop=True)

# GNN Analysis

In [ ]:
timestamps = np.load(f'{data_dir}/demand_graph_timestamps.pkl.npz')
stamps = [timestamps[f'arr_{i}'] for i in range(8757, len(timestamps)-1)]
gnn_embedding = torch.load(f'gnn_cnn_2024-11-02 14_35_41.074099_output_embedding.pt')

In [ ]:
station_clusters = [
    74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,
    87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99,
    100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
    113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
    126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
    139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151,
    152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164,
    165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177,
    178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190,
    191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203,
    205, 206, 208, 209, 210, 211, 212, 213, 214, 216, 217, 218, 219,
    220, 222, 223, 224, 225, 226, 227, 228, 230, 231, 232, 233, 234,
    235, 237, 238, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249,
    250, 251, 252, 253, 255, 257, 258, 261, 264, 265, 266, 274, 275,
    278
]





In [ ]:
len(gnn_embedding)

10942

In [ ]:
datetimes = [datetime(int(arr[0][0]), int(arr[0][1]), int(arr[0][2]), int(arr[0][3])) for arr in stamps]
total_demand = [int(sum(arr[:,4])) for arr in stamps]
demand_degrees = [int(sum(arr[:,4]>0)) for arr in stamps]
mean_embeddings = [emb.mean(dim=0) for emb in gnn_embedding]
variance_embeddings = [emb.var(dim=0) for emb in gnn_embedding]

## Mean and Variance

In [ ]:
df_embedding = pd.DataFrame({
    'datetime': datetimes,
    'total_demand_1h': total_demand,
    'demand_degrees_1h': demand_degrees,
    **{f"dim_mean_{i}": [emb[i].item() for emb in mean_embeddings] for i in range(mean_embeddings[0].size(0))},
    **{f"dim_var_{i}": [emb[i].item() for emb in variance_embeddings] for i in range(variance_embeddings[0].size(0))}
})

In [ ]:
df_embedding['datetime'] = df_embedding['datetime'] + timedelta(hours=1)

## Clustering

In [ ]:
df_gnn_clusters = pd.DataFrame()
for ii in range(len(gnn_embedding)):
  kmeans = KMeans(n_clusters=4, random_state=42)
  labels = kmeans.fit_predict(gnn_embedding[ii].detach().numpy())
  df_current = pd.DataFrame({
      "datetime": datetimes[ii],
      "gnn_label_cluster": labels,
      "start_station_cluster": station_clusters

  })
  df_gnn_clusters = pd.concat([df_gnn_clusters, df_current])

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


In [ ]:
import plotly.express as px
def get_cluster_screenshots(time='2023-01-2 9:00:00'):
  standard_stations = pd.read_csv(f"{data_dir}/standard_stations.csv")
  cluster_example = df_gnn_clusters[df_gnn_clusters['datetime'] == time]
  standard_stations = standard_stations.drop_duplicates(subset=['cluster_stations'])
  cluster_example = cluster_example.merge(standard_stations[[
    'cluster_stations',
    'station_latitude_centroid_stations',
    'station_longitude_centroid_stations'
  ]], left_on=['start_station_cluster'], right_on=['cluster_stations'])

  cluster_example['gnn_label_cluster'] = cluster_example['gnn_label_cluster'].astype(str)
  fig = px.scatter_mapbox(cluster_example, lat='station_latitude_centroid_stations', lon='station_longitude_centroid_stations',
                          color = "gnn_label_cluster",
                          mapbox_style="open-street-map",
                          zoom=10.8
  ).update_traces(marker={"size": 10})

  return fig

In [ ]:
get_cluster_screenshots(time='2023-01-2 9:00:00')

In [ ]:
get_cluster_screenshots(time='2023-01-2 10:00:00')

In [ ]:
get_cluster_screenshots(time='2023-01-2 11:00:00')

In [ ]:

get_cluster_screenshots(time='2023-01-2 12:00:00')

In [ ]:
df_gnn_clusters = df_gnn_clusters.rename(
    columns={"datetime": "started_at_hourly"}
).merge(df_concatenated[['demand', 'started_at_hourly', 'start_station_cluster']], on=['started_at_hourly', 'start_station_cluster'],
how="left").dropna()

df_concatenated = df_concatenated.merge(df_gnn_clusters.rename(
    columns={"datetime": "started_at_hourly"})[[
        'started_at_hourly',
        'start_station_cluster',
        'gnn_label_cluster']],
    on=['started_at_hourly', 'start_station_cluster'],
how="left")

cluster_aggregate = df_gnn_clusters.groupby(["gnn_label_cluster", "started_at_hourly"]).agg({
    "demand": ["mean", "sum"]
})
cluster_aggregate.columns = cluster_aggregate.columns.droplevel()

cluster_aggregate = cluster_aggregate.reset_index().rename(columns={
    "mean": "mean_gnn_cluster_demand_1h",
    "sum": "total_gnn_cluster_demand_1h"
})

cluster_aggregate['started_at_hourly'] = cluster_aggregate['started_at_hourly'] + timedelta(hours=1)

In [ ]:
df_concatenated = df_concatenated.merge(cluster_aggregate,
    on=['gnn_label_cluster', 'started_at_hourly'],
how="left"
)

df_concatenated['mean_gnn_cluster_demand_1h'] = df_concatenated['mean_gnn_cluster_demand_1h'].fillna(1)
df_concatenated['total_gnn_cluster_demand_1h'] = df_concatenated['total_gnn_cluster_demand_1h'].fillna(1)

## Cosine Similarity

In [ ]:
similarity_cols = [f"cosine_sim_{col}" for col in range(gnn_embedding[0].detach().numpy().shape[0])]
df_cosine = pd.DataFrame()
for ii in range(len(gnn_embedding)):

  similarity = cosine_similarity(gnn_embedding[ii].detach().numpy())
  df_current = pd.DataFrame({
      "datetime": datetimes[ii],
      "start_station_cluster": station_clusters
  })

  similarity_df = pd.DataFrame(similarity, columns=similarity_cols)
  df_current = pd.concat([df_current, similarity_df], axis=1)
  df_cosine = pd.concat([df_cosine, df_current])

In [ ]:
del demand
del datetimes
del total_demand
del gnn_embedding
del demand_degrees
del mean_embeddings
del variance_embeddings
del df_gnn_clusters
del cluster_aggregate

In [ ]:
df_cosine['datetime'] = df_cosine['datetime'] + timedelta(hours=1)

df_cosine = df_cosine.rename(
    columns={"datetime": "started_at_hourly"}
)

df_cosine.to_csv(f'{data_dir}/cosine_similarity.csv')

In [ ]:
del df_cosine

In [ ]:
df_concatenated = df_concatenated.merge(df_embedding.rename(
    columns={"datetime": "started_at_hourly"}), on=['started_at_hourly'], how="left")
df_concatenated = df_concatenated.fillna(0)

In [ ]:
df_concatenated#.drop_duplicates(subset=['start_station_cluster', 'started_at_hourly'])

,start_station_cluster,started_at_year,started_at_month,started_at_day,started_at_hour,demand,started_at_week,started_at_quarter,started_at_dayofweek,started_at_daily,clusters,started_at_hourly,is_holiday,temp,dwpt,rhum,prcp,wdir,wspd,pres,coco,demand_lag_1_h,demand_lag_2_h,demand_lag_24_h,temp_lag_1_h,temp_lag_2_h,temp_lag_24_h,prcp_lag_1_h,prcp_lag_2_h,prcp_lag_24_h,rhum_lag_1_h,rhum_lag_2_h,rhum_lag_24_h,wspd_lag_1_h,wspd_lag_2_h,wspd_lag_24_h,coco_lag_1_h,coco_lag_2_h,coco_lag_24_h,gnn_label_cluster,mean_gnn_cluster_demand_1h,total_gnn_cluster_demand_1h,total_demand_1h,demand_degrees_1h,dim_mean_0,dim_mean_1,dim_mean_2,dim_mean_3,dim_mean_4,dim_mean_5,dim_mean_6,dim_mean_7,dim_mean_8,dim_mean_9,dim_mean_10,dim_mean_11,dim_mean_12,dim_mean_13,dim_mean_14,dim_mean_15,dim_mean_16,dim_mean_17,dim_mean_18,dim_mean_19,dim_mean_20,dim_mean_21,dim_mean_22,dim_mean_23,dim_mean_24,dim_mean_25,dim_mean_26,dim_mean_27,dim_mean_28,dim_mean_29,dim_mean_30,dim_mean_31,dim_mean_32,dim_mean_33,dim_mean_34,dim_mean_35,dim_mean_36,dim_mean_37,dim_mean_38,dim_mean_39,dim_mean_40,dim_mean_41,dim_mean_42,dim_mean_43,dim_mean_44,dim_mean_45,dim_mean_46,dim_mean_47,dim_mean_48,dim_mean_49,dim_var_0,dim_var_1,dim_var_2,dim_var_3,dim_var_4,dim_var_5,dim_var_6,dim_var_7,dim_var_8,dim_var_9,dim_var_10,dim_var_11,dim_var_12,dim_var_13,dim_var_14,dim_var_15,dim_var_16,dim_var_17,dim_var_18,dim_var_19,dim_var_20,dim_var_21,dim_var_22,dim_var_23,dim_var_24,dim_var_25,dim_var_26,dim_var_27,dim_var_28,dim_var_29,dim_var_30,dim_var_31,dim_var_32,dim_var_33,dim_var_34,dim_var_35,dim_var_36,dim_var_37,dim_var_38,dim_var_39,dim_var_40,dim_var_41,dim_var_42,dim_var_43,dim_var_44,dim_var_45,dim_var_46,dim_var_47,dim_var_48,dim_var_49
0,74,2022,12,31,0,0.0,52,4,5,2022-12-31,1,2022-12-31 00:00:00,False,16.1,14.5,90.0,0.0,220.0,31.7,1012.7,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00
1,74,2022,12,31,1,0.0,52,4,5,2022-12-31,1,2022-12-31 01:00:00,False,15.6,14.5,93.0,0.3,240.0,40.7,1012.6,7,0.0,0.0,0.0,16.1,0.0,0.0,0.0,0.0,0.0,90.0,0.0,0.0,31.7,0.0,0.0,7.0,0.0,0.0,0.0,1.000000,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000

# Scaling

In [ ]:
#df_concatenated.to_csv(f'{data_dir}/final_model_input_unscaled.csv')

In [ ]:
le = preprocessing.LabelEncoder()

scalers = {}

SCALER_DATETIME_FEATURES = [
    'started_at_year',
    'started_at_month',
    'started_at_day',
    'started_at_hour',
    'started_at_week',
    'started_at_quarter',
    'started_at_dayofweek',
]

SCALER_DEMAND_FEATURES = [
    "demand_lag_1_h",
    "demand_lag_2_h",
    "demand_lag_24_h",
    "mean_gnn_cluster_demand_1h",
    "total_gnn_cluster_demand_1h",
    "total_demand_1h",
    "demand_degrees_1h"
]

SCALER_WEATHER_FEATURES = [
    "temp",
    "dwpt",
    "rhum",
    "prcp",
    "wdir",
    "wspd",
    "pres",
    "coco",
    "temp_lag_1_h",
    "temp_lag_2_h",
    "temp_lag_24_h",
    "prcp_lag_1_h",
    "prcp_lag_2_h",
    "prcp_lag_24_h",
    "rhum_lag_1_h",
    "rhum_lag_2_h",
    "rhum_lag_24_h",
    "wspd_lag_1_h",
    "wspd_lag_2_h",
    "wspd_lag_24_h",
    "coco_lag_1_h",
    "coco_lag_2_h",
    "coco_lag_24_h"
]


SCALER_FEATURES = [
    "temp",
    "dwpt",
    "rhum",
    "prcp",
    "wdir",
    "wspd",
    "pres",
    "coco",
    "temp_lag_1_h",
    "temp_lag_2_h",
    "temp_lag_24_h",
    "prcp_lag_1_h",
    "prcp_lag_2_h",
    "prcp_lag_24_h",
    "rhum_lag_1_h",
    "rhum_lag_2_h",
    "rhum_lag_24_h",
    "wspd_lag_1_h",
    "wspd_lag_2_h",
    "wspd_lag_24_h",
    "demand_lag_1_h",
    "demand_lag_2_h",
    "demand_lag_24_h",
    "mean_gnn_cluster_demand_1h",
    "total_gnn_cluster_demand_1h",
    "total_demand_1h",
    "demand_degrees_1h",
    'started_at_year',
    'started_at_month', 'started_at_day',
    'started_at_hour',
    'started_at_week',
    'started_at_quarter',
    'started_at_dayofweek',
]

target_scaler = MinMaxScaler(feature_range=(1, 2))

demand_features_scaler = MinMaxScaler(feature_range=(1, 2))
weather_scaler = MinMaxScaler(feature_range=(1, 2))
datetime_features_scaler = MinMaxScaler(feature_range=(1, 2))
#start_station_cluster_scaler = MinMaxScaler(feature_range=(1, 2))


#scaler = MinMaxScaler(feature_range=(1, 2))
#df_concatenated["start_station_cluster"] = start_station_cluster_scaler.fit_transform(df_concatenated[['start_station_cluster']])
df_concatenated[SCALER_DEMAND_FEATURES] = demand_features_scaler.fit_transform(df_concatenated[SCALER_DEMAND_FEATURES])
df_concatenated[SCALER_WEATHER_FEATURES] = weather_scaler.fit_transform(df_concatenated[SCALER_WEATHER_FEATURES])
df_concatenated[SCALER_DATETIME_FEATURES] = datetime_features_scaler.fit_transform(df_concatenated[SCALER_DATETIME_FEATURES])

#scalers[feature] = scaler

df_concatenated[['demand_target']] = target_scaler.fit_transform(df_concatenated[['demand']])
df_concatenated['start_station_cluster'] = le.fit_transform(df_concatenated['start_station_cluster'])
df_concatenated['is_holiday'] = df_concatenated['is_holiday'].astype(int)

filename = f'{models_dir}/target_scaler.sav'
joblib.dump(target_scaler, filename)

filename = f'{models_dir}/datetime_features_scaler.sav'
joblib.dump(datetime_features_scaler, filename)

filename = f'{models_dir}/demand_features_scaler.sav'
joblib.dump(demand_features_scaler, filename)

filename = f'{models_dir}/weather_scaler.sav'
joblib.dump(weather_scaler, filename)

#filename = f'{models_dir}/start_station_cluster_scaler.sav'
#joblib.dump(start_station_cluster_scaler, filename)

['/content/drive/MyDrive/Thesis/models/weather_scaler.sav']

In [ ]:

filename = f'{models_dir}/start_station_encoder.sav'
joblib.dump(le, filename)

['/content/drive/MyDrive/Thesis/models/start_station_encoder.sav']

In [ ]:
#df_concatenated.to_csv(f'{data_dir}/final_model_input_partial_scale_4.csv')

In [ ]:
df_concatenated['demand'].value_counts()

,count
demand,
0.0,782837
1.0,229889
2.0,120433
3.0,73065
4.0,48252
5.0,34032
6.0,24821
7.0,18673
8.0,14570
